# Fetch Leaderboard Data

Goals:
* Request position data from The Graph
* Rank users on their profit and loss within a specified time period

In [124]:
import requests
import pandas as pd
import json
from decimal import Decimal


In [127]:
# constants
FUTURES_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-kovan-futures'



In [128]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)

In [161]:
# queries
userOpenPositions = """
{{
  futuresPositions(
    where: {{
      account: "{address}"
      isOpen: true
    }}
  ) {{
    id
    timestamp
    market
    asset
    isOpen
    isLiquidated
    size
    margin 
    pnl
    feesPaid
    account
    size
    lastPrice
    initialMargin
    netTransfers
    totalDeposits
  }}
}}
"""

userOpenPositionsBlock = """
{{
  futuresPositions(
    where: {{
      account: "{address}"
      isOpen: true
    }}
    block: {{
      number: {block}
    }}
  ) {{
    id
    timestamp
    market
    asset
    isOpen
    isLiquidated
    size
    margin 
    pnl
    feesPaid
    account
    size
    lastPrice
    initialMargin
    netTransfers
    totalDeposits
  }}
}}
"""

userTrades = """
{{
  futuresTrades(
    where: {{
      account: "{address}"
    }}
    orderBy: timestamp
    orderDirection: desc
  ) {{
    id
    timestamp
    account
    size
    asset
    price
    feesPaid
    pnl
    orderType
    positionId
    positionSize
    positionClosed
  }}
}}
"""


QUERIES = {
    'userTrades': userTrades,
    'userOpenPositions': userOpenPositions,
    'userOpenPositionsBlock': userOpenPositionsBlock
}

In [162]:
## trades
# query trades
query = QUERIES['userTrades'].format(
    address="0x3239a95A9262034ca28b9a03133775f716f119f8")

r = requests.post(FUTURES_ENDPOINT, json={'query': query})
if r.status_code == 200:
    responseJson = json.loads(r.text)
    df_trades = pd.DataFrame(responseJson['data']['futuresTrades'])
else:
    print(r.text)


In [163]:
# convert decimal columns
decimal_cols = [
    'size',
    'price',
    'feesPaid',
    'pnl',
    'positionSize'
]
for col in decimal_cols:
    df_trades[col] = df_trades[col].apply(convertDecimals)

# inspect the data
df_trades.head()


,id,timestamp,account,size,asset,price,feesPaid,pnl,orderType,positionId,positionSize,positionClosed
0,0x68195320d477d393101d5ffd7390fdcbf7247530e7ea...,1654796733,0x3239a95a9262034ca28b9a03133775f716f119f8,1,0x73455448000000000000000000000000000000000000...,1800.64787528,3.60129575056,0,Market,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbd,1,False
1,0xd9ca4a30bdc9ad629e3d41bf5fa8d0d73464fd579610...,1654795789,0x3239a95a9262034ca28b9a03133775f716f119f8,-0.5590214012877607,0x73455448000000000000000000000000000000000000...,1800.64787528,1.00660069846485456,0,NextPrice,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbc,0,True
2,0x7900262c0838117910aa3a031d93eb50623c3a290c74...,1654787534,0x3239a95a9262034ca28b9a03133775f716f119f8,0.5590214012877607,0x73455448000000000000000000000000000000000000...,1788.84027999,1.999999999999999994,0,Market,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbc,0.5590214012877607,False
3,0x8a2db0bdd1bee36322d10807c26c34b81e5dbc04a2e7...,1654638570,0x3239a95a9262034ca28b9a03133775f716f119f8,3.8481301507261787,0x73455448000000000000000000000000000000000000...,1840.8376,14.16756534230083411,3.842919936427301483,Market,0x698e403aac625345c6e5fc2d0042274350bedf78-0xba,0,True
4,0x6a2230bdfa371d8ed9f669a23bc35d03f1faa33f95a0...,1654637222,0x3239a95a9262034ca28b9a03133775f716f119f8,0.5444495061082703,0x73455448000000000000000000000000000000000000...,1841.83624604,2.005573668977577238,-225.410698420415352431,Market,0x698e403aac625345c6e5fc2d0042274350bedf78-0xba,-3.8481301507261787,False


In [164]:
## positions
# query positions
query = QUERIES['userOpenPositions'].format(
    address="0x3239a95A9262034ca28b9a03133775f716f119f8")

r = requests.post(FUTURES_ENDPOINT, json={'query': query})
if r.status_code == 200:
    responseJson = json.loads(r.text)
    df_positions = pd.DataFrame(responseJson['data']['futuresPositions'])
else:
    print(r.text)


In [165]:
df_positions

,id,timestamp,market,asset,isOpen,isLiquidated,size,margin,pnl,feesPaid,account,lastPrice,initialMargin,netTransfers,totalDeposits
0,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbd,1654803246,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,True,False,1000000000000000000,246456851862654048901,-11034748430000000000,3601295750560000000,0x3239a95a9262034ca28b9a03133775f716f119f8,1789613126850000000000,293554997087337311309,0,0


In [166]:
# convert decimal columns
decimal_cols = [
    'size',
    'margin',
    'feesPaid',
    'pnl',
    'lastPrice',
    'initialMargin',
    'netTransfers',
    'totalDeposits'
]
for col in decimal_cols:
    df_positions[col] = df_positions[col].apply(convertDecimals)

# inspect the data
df_positions.head()


,id,timestamp,market,asset,isOpen,isLiquidated,size,margin,pnl,feesPaid,account,lastPrice,initialMargin,netTransfers,totalDeposits
0,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbd,1654803246,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,True,False,1,246.456851862654048901,-11.03474843,3.60129575056,0x3239a95a9262034ca28b9a03133775f716f119f8,1789.61312685,293.554997087337311309,0,0


### Query by Block

In [167]:
## blocks
deposit_block = 3803686
open_block = 3803886
close_block = 3804059




In [175]:
## open positions at block
# query positions
query = QUERIES['userOpenPositionsBlock'].format(
    address="0x268671dd4Dc2417b56EDDca2eDb6b2D07e8Ffb13",
    block=open_block
)

r = requests.post(FUTURES_ENDPOINT, json={'query': query})
if r.status_code == 200:
    responseJson = json.loads(r.text)
    df_positions = pd.DataFrame(responseJson['data']['futuresPositions'])
else:
    print(r.text)

# convert decimal columns
if(df_positions.shape[0] > 0):
    decimal_cols = [
        'size',
        'margin',
        'feesPaid',
        'pnl',
        'lastPrice',
        'initialMargin',
        'netTransfers',
        'totalDeposits'
    ]
    for col in decimal_cols:
        df_positions[col] = df_positions[col].apply(convertDecimals)

# inspect the data
df_positions.head()


,id,timestamp,market,asset,isOpen,isLiquidated,size,margin,pnl,feesPaid,account,lastPrice,initialMargin,netTransfers,totalDeposits
0,0x698e403aac625345c6e5fc2d0042274350bedf78-0xbb,1654727636,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,True,False,-0.5540135511714617,96.999999999999999751,0,3.000000000000000249,0x268671dd4dc2417b56eddca2edb6b2d07e8ffb13,1805.01,96.999999999999999751,0,0
